In [1]:
import gymnasium as gym
import torch
import torch.optim as optim
import numpy as np
import time

import matplotlib.pyplot as plt

import my_package
from my_package.core.AC_v4 import AC_agent

/home/paolo/.local/lib/python3.6/site-packages/gymnasium/core.py:27: UserWarning: WARN: Gymnasium minimally supports python 3.6 as the python foundation not longer supports the version, please update your version to 3.7+
  "Gymnasium minimally supports python 3.6 as the python foundation not longer supports the version, please update your version to 3.7+"


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

env_name = 'ShipRovCont-v0'
env_options = {
    'generate_random_ship':             True,
    'workspace_safe_distance':          5,
    'ship_scale_factor':                2.0,
    'segments_lenght':                  0.25,
    'frame_per_step':                   10, # 2Hz
    'init_pose':                        None,
    'agent_radius':                     0.75,     # Ho aumentato l' agent radius in training
    'v_surge_max':                      0.1,
    'v_sway_max' :                      0.05,
    'yaw_rel_max':                      np.pi/4,
    'frontal_safe_distance':            0.5,
    'lidar_params':                     {'n_beams': 10, 'max_range': 5.0, 'FoV': np.pi/2},
    'draw_lidar':                       True,
    'max_steps':                        2000
}	

""" Init env """
env = gym.make(env_name, Options=env_options, workspace=(0,0,20,20))
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.shape[0]

max_episodes = 5
data_window = 1

buffer_params       = {'buffer_size': 1000000, 'batch_size': 256}
AC_params           = {'learn_every': 16, 'n_learn': 8, 'gamma': 0.99, 'tau': 1e-3, 'lr_actor': 1e-4, 'lr_critic': 1e-4}
noise_type          = 'param'
noise_params        = {'desired_std': 0.5, 'scalar': 0.05, 'scalar_decay': 0.99, 'min_std': 0.01, 'decay_with_ep': True}

""" Init AC agent """
agent = AC_agent(
    state_size=state_dim,
    action_size=action_dim,
    max_episodes=max_episodes,
    device=device,
    noise_params=noise_params,
    noise_type=noise_type,
    AC_params=AC_params,
    buffer_params=buffer_params
)

""" Data collect variables """
total_rewards = np.zeros(max_episodes)
noise_history = np.zeros(max_episodes)
len_episodes = np.zeros(max_episodes)
coverage_per_ep = np.zeros(max_episodes)

/home/paolo/miniconda3/envs/zenoenv/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: Your system is avx2 capable but pygame was not built with support for it. The performance of some of your blits could be adversely affected. Consider enabling compile time detection with environment variables like PYGAME_DETECT_AVX2=1 if you are compiling without cross compilation.
  return f(*args, **kwds)


RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [3]:
start_time = time.time()

for episode in range(max_episodes):
    state, info = env.reset()
    agent.reset(episode)
    done = False
    
    while not done:
        action = agent.act(state)
        next_state, reward, terminated, truncated, info = env.step(action)
        agent.step(state, action, reward, next_state, terminated)

        total_rewards[episode] += reward
        len_episodes[episode] += 1
        state = next_state
        done = terminated or truncated

    coverage_per_ep[episode] = info['coverage']
    print(info)
    if episode % data_window == 0 and episode != 0:
        ma_reward = np.mean(total_rewards[episode-data_window:episode])
        ma_coverage = np.mean(coverage_per_ep[episode-data_window:episode])
        print(f"Ep {episode}/{max_episodes}, MA Reward: {ma_reward:.2f}, MA Coverage: {ma_coverage:.1f} %")

env.close()

total_time_seconds = time.time() - start_time
hours = int(total_time_seconds // 3600)
minutes = int((total_time_seconds % 3600) // 60)
seconds = int(total_time_seconds % 60)
print(f"Training finito in: {hours} ore {minutes} minuti e {seconds} secondi")

NameError: name 'agent' is not defined

In [ ]:
len_episodes